In [26]:
import pandas as pd

import ast

In [27]:
foodRaw_df = pd.read_csv('../../Resources/CSVs/all_menu_items.csv')

foodRaw_df.head()

,Restaurant,Food_Name,Nutrients,Calories
0,Subway,Cheese Pizza,"[{'value': 28, 'attr_id': 203}, {'value': 23, ...",720
1,Subway,Sausage Pizza,"[{'value': 35, 'attr_id': 203}, {'value': 35, ...",860
2,Subway,Meatball Pizza,"[{'value': 37, 'attr_id': 203}, {'value': 33, ...",860
3,Subway,Pepperoni Pizza,"[{'value': 35, 'attr_id': 203}, {'value': 33, ...",840
4,Subway,Bacon Pizza,"[{'value': 37, 'attr_id': 203}, {'value': 31, ...",840


In [28]:
nutrient_map = pd.read_csv('../../Resources/CSVs/Nutritionix API v2 - Full Nutrient USDA Field Mapping.csv')

nutrient_map = nutrient_map.set_index('attr_id')

nutrient_map.head()

,2018 NFP,usda_tag,name,unit,notes,bulk_csv_field,cxh_attribute_name,Unnamed: 8
attr_id,,,,,,,,
203,1.0,PROCNT,Protein,g,NaN,nf_protein,PRO-,NaN
204,1.0,FAT,Total lipid (fat),g,NaN,nf_total_fat,FAT,NaN
205,1.0,CHOCDF,"Carbohydrate, by difference",g,NaN,nf_total_carbohydrate,CHO-,NaN
207,0.0,ASH,Ash,g,NaN,NaN,NaN,NaN
208,1.0,ENERC_KCAL,Energy,kcal,NaN,nf_calories,ENER-,NaN


In [29]:
for index,row in foodRaw_df.iterrows(): #iterate each index and row.
    
    nutrients = ast.literal_eval(row['Nutrients']) #The original column was as string. ast.literal_eval -->interprets "strings, bytes, numbers, tuples, lists, dicts, sets, booleans, and None." and returns it in the right format.. Anything else is an error.
    
    for nutrient in nutrients: #Iterates each of the nutrients dictionaries. 
        
        name = nutrient_map.loc[nutrient['attr_id'],'name'] #locates the name in the nutrient_map csv. 
        
        conversion_dict = { #Converts the columns from grams to calories.
            'Protein' : 4, #To cal
            'Carbohydrate, by difference' : 4, #to cal
            'Total lipid (fat)' : 9, #to cal
            'Vitamin A, IU' : 0.3, #to mcg RAE
            'Fatty acids, total trans' : 10,
            'Fatty acids, total saturated' : 9,
            'Sugars, total' : 4} 
        
        if name in conversion_dict.keys():
            conversionFactor = conversion_dict[name]
        else:
            conversionFactor = 1
        
        value = nutrient['value'] 
        
       
        if name in foodRaw_df.columns: #if the nutrient is a DataFrame column.
                
            foodRaw_df.loc[index,name] = value * conversionFactor #Adds the value. 
        
        else: #if the nutrient is not in the DataFrame
            foodRaw_df[name] =''
            foodRaw_df.loc[index,name] = value * conversionFactor
        
food_df = foodRaw_df.replace(r"^\s*$", 0, regex = True) #Finds and replace empty cells with 0. empty cell pattern = r"^\s*$"

food_df.head()

,Restaurant,Food_Name,Nutrients,Calories,Protein,Total lipid (fat),"Carbohydrate, by difference",Energy,"Sugars, total","Fiber, total dietary","Calcium, Ca","Iron, Fe","Sodium, Na","Vitamin A, IU","Vitamin C, total ascorbic acid",Cholesterol,"Fatty acids, total trans","Fatty acids, total saturated"
0,Subway,Cheese Pizza,"[{'value': 28, 'attr_id': 203}, {'value': 23, ...",720,112,207.0,388,720,28,4,520,7.2,1530,300.0,9.0,50,5.0,81.0
1,Subway,Sausage Pizza,"[{'value': 35, 'attr_id': 203}, {'value': 35, ...",860,140,315.0,392,860,32,4,650,7.2,1870,300.0,9.0,80,5.0,126.0
2,Subway,Meatball Pizza,"[{'value': 37, 'attr_id': 203}, {'value': 33, ...",860,148,297.0,400,860,28,5,650,8.1,1840,300.0,9.0,80,10.0,126.0
3,Subway,Pepperoni Pizza,"[{'value': 35, 'attr_id': 203}, {'value': 33, ...",840,140,297.0,388,840,32,4,650,7.2,1800,300.0,9.0,70,10.0,117.0
4,Subway,Bacon Pizza,"[{'value': 37, 'attr_id': 203}, {'value': 31, ...",840,148,279.0,392,840,32,4,650,7.2,2060,300.0,9.0,70,5.0,108.0


In [30]:
#DataFrame columns order edit and columns rename.

foodFiltered_df = food_df[['Restaurant',
                           'Food_Name',
                           'Calories',
                           'Protein',
                           'Carbohydrate, by difference',
                           'Fiber, total dietary',
                           'Sugars, total',
                           'Total lipid (fat)',
                           'Fatty acids, total trans',
                           'Fatty acids, total saturated',
                           'Cholesterol',
                           'Calcium, Ca',
                           'Iron, Fe',
                           'Sodium, Na',
                           'Vitamin A, IU',
                          'Vitamin C, total ascorbic acid']]

columns_name = {
    'Protein' : 'Protein (Cal)',
    'Carbohydrate, by difference' : 'Carbohydrate (Cal)',
    'Fiber, total dietary' : 'Fiber (gr)',
    'Sugars, total' : 'Total Sugar (Cal)',
    'Total lipid (fat)' : 'Total Fat (Cal)',
    'Cholesterol' : 'Cholesterol (mg)',
    'Calcium, Ca' : 'Calcium (mg)',
    'Iron, Fe' : 'Iron (mg)',
    'Sodium, Na' : 'Sodium (mg)',
    'Vitamin A, IU' : 'Vitamin A (mcg RAE)',
    'Vitamin C, total ascorbic acid' : 'Vitamin C (mg)',
    'Fatty acids, total trans' : 'Total Trans Fat (Cal)',
    'Fatty acids, total saturated' : 'Total Saturated Fat (Cal)'
    
}

foodFiltered_df = foodFiltered_df.rename(columns = columns_name)

foodFiltered_df.head()


,Restaurant,Food_Name,Calories,Protein (Cal),Carbohydrate (Cal),Fiber (gr),Total Sugar (Cal),Total Fat (Cal),Total Trans Fat (Cal),Total Saturated Fat (Cal),Cholesterol (mg),Calcium (mg),Iron (mg),Sodium (mg),Vitamin A (mcg RAE),Vitamin C (mg)
0,Subway,Cheese Pizza,720,112,388,4,28,207.0,5.0,81.0,50,520,7.2,1530,300.0,9.0
1,Subway,Sausage Pizza,860,140,392,4,32,315.0,5.0,126.0,80,650,7.2,1870,300.0,9.0
2,Subway,Meatball Pizza,860,148,400,5,28,297.0,10.0,126.0,80,650,8.1,1840,300.0,9.0
3,Subway,Pepperoni Pizza,840,140,388,4,32,297.0,10.0,117.0,70,650,7.2,1800,300.0,9.0
4,Subway,Bacon Pizza,840,148,392,4,32,279.0,5.0,108.0,70,650,7.2,2060,300.0,9.0


In [31]:
foodFiltered_df[foodFiltered_df['Restaurant']== 'Dairy Queen']

,Restaurant,Food_Name,Calories,Protein (Cal),Carbohydrate (Cal),Fiber (gr),Total Sugar (Cal),Total Fat (Cal),Total Trans Fat (Cal),Total Saturated Fat (Cal),Cholesterol (mg),Calcium (mg),Iron (mg),Sodium (mg),Vitamin A (mcg RAE),Vitamin C (mg)
410,Dairy Queen,"DQ Treatzza Pizza, Heath",190,8,104,1,76,81.0,0.0,54.0,5,0,0.0,90,0.0,0.0
411,Dairy Queen,"DQ Treatzza Pizza, Choco Brownie",190,12,104,1,72,90.0,0.0,54.0,10,0,0.0,100,0.0,0.0
412,Dairy Queen,"DQ Treatzza Pizza, M&M'sChocolate Candies",200,12,112,1,80,81.0,0.0,63.0,5,0,0.0,85,0.0,0.0
413,Dairy Queen,"DQ Treatzza Pizza, Reese'sPeanut Butter Cup",200,12,96,1,68,90.0,0.0,54.0,5,0,0.0,110,0.0,0.0
414,Dairy Queen,Kids Hamburger,320,72,124,1,28,126.0,5.0,54.0,50,0,0.0,620,0.0,0.0
415,Dairy Queen,DQ Ultimate Burger,720,176,120,1,24,423.0,20.0,171.0,145,0,0.0,1390,0.0,0.0
416,Dairy Queen,Side Salad,25,4,20,2,12,0.0,0.0,0.0,0,0,0.0,15,0.0,0.0
417,Dairy Queen,"Chicken BLT Salad, Grilled",270,140,40,3,20,99.0,0.0,45.0,80,0,0.0,1090,0.0,0.0
418,Dairy Queen,"Chicken BLT Salad, Crispy",400,104,112,4,20,189.0,0.0,54.0,60,0,0.0,1070,0.0,0.0
419,Dairy Queen,$5 Buck Lunch Crispy Chicken Salad,310,76,92,3,8,144.0,0.0,40.5,45,0,0.0,770,0.0,0.0


In [32]:
#Load kcal requirments from FDA

calorieNeeds_male = pd.read_excel('../../FDA Tables XLSX/Appendix 2. Estimated Calorie Needs -2020 Dietary Guidelines _ health.gov_0002.xlsx', sheet_name = 'Males')

calorieNeeds_male = calorieNeeds_male[calorieNeeds_male['AGE'] >= 14]

calorieNeeds_male = calorieNeeds_male.reset_index(drop= True)

calorieNeeds_male

,AGE,Sedentary,Moderately active,Active
0,14,2000,2400,2800
1,15,2200,2600,3000
2,16,2400,2800,3200
3,17,2400,2800,3200
4,18,2400,2800,3200
...,...,...,...,...
58,72,2000,2200,2600
59,73,2000,2200,2600
60,74,2000,2200,2600
61,75,2000,2200,2600


In [33]:
#Health Parameters Male

nutritionParameters_dict = {'Protein (Cal)' : ((12 + 3) / 2)/100,
                           'Carbohydrate (Cal)' : ((10 + 25) / 2) / 100,
                           'Fiber (gr)' : (8 + 30) / 2,
                           'Total Sugar (Cal)': .04, #https://www.livestrong.com/article/333407-how-many-calories-should-i-eat-at-lunch/
                           'Total lipid (Cal)': ((12 + 8) / 2)/ 100,
                           'Total Trans Fat (Cal)': .01, #https://medlineplus.gov/ency/patientinstructions/000786.htm#:~:text=You%20should%20limit%20saturated%20fat,or%202%20grams%20per%20day.
                           'Total Saturated Fat (Cal)': 0.03,
                           'Cholesterol (mg)' : 100, #https://www.healthline.com/health/high-cholesterol/rda#:~:text=Following%20dietary%20guidelines%2C%20doctors%20used,high%20risk%20of%20heart%20disease.
                           'Calcium (mg)': (835 + 330) / 2,
                           'Iron (mg)': (5 + 15) / 2,
                           'Sodium, (mg)': (770 + 150)/2,
                           'Vitamin A (mcg RAE)': (1000 + 230) / 2,
                            'Vitamin C (mg)' : 21,
                            'Calories per Meal' : 1/3}
    


maleNutrition19to30_dict = {'Protein (Cal)' : 0.225/3,
                           'Carbohydrate (Cal)' : 0.175,
                           'Fiber (gr)' : 33.6/3,
                           'Total Sugar (gr)': 40/3, #https://www.livestrong.com/article/333407-how-many-calories-should-i-eat-at-lunch/
                           'Total lipid (Cal)': 0.176/3,
                           'Total Trans Fat (Cal)': .01/3, #https://medlineplus.gov/ency/patientinstructions/000786.htm#:~:text=You%20should%20limit%20saturated%20fat,or%202%20grams%20per%20day.
                           'Total Saturated Fat (Cal)': 0.1/3,
                           'Cholesterol (mg)' : 250/3, #https://www.healthline.com/health/high-cholesterol/rda#:~:text=Following%20dietary%20guidelines%2C%20doctors%20used,high%20risk%20of%20heart%20disease.
                           'Calcium (mg)': 1000/3,
                           'Iron (mg)': 8/2,
                           'Sodium, (mg)': 2300/3,
                           'Vitamin A, IU': 900/3,
                            'Vitamin C (mg)' : 90/3}
    
maleNutrition31to50_dict = {'Protein (Cal)' : 0.225/3,
                           'Carbohydrate (Cal)' : 0.175,
                           'Fiber (gr)' : 30.8/3,
                           'Total Sugar (gr)': 40/3, #https://www.livestrong.com/article/333407-how-many-calories-should-i-eat-at-lunch/
                           'Total lipid (Cal)': 0.176/3,
                            'Total Trans Fat (Cal)': .01/3, #https://medlineplus.gov/ency/patientinstructions/000786.htm#:~:text=You%20should%20limit%20saturated%20fat,or%202%20grams%20per%20day.
                           'Total Saturated Fat (Cal)': 0.1/3,
                           'Cholesterol (mg)' : 250/3, #https://www.healthline.com/health/high-cholesterol/rda#:~:text=Following%20dietary%20guidelines%2C%20doctors%20used,high%20risk%20of%20heart%20disease.
                           'Calcium (mg)': 1000/3,
                           'Iron (mg)': 8/2,
                           'Sodium (mg)': 2300/3,
                           'Vitamin A (mcg RAE)': 900/3,
                            'Vitamin C (mg)' : 90/3}
    
maleNutrition51_dict = {'Protein (Cal)' : 0.225/3,
                        'Carbohydrate (Cal)' : 0.175,
                        'Fiber (gr)' : 28/3,
                        'Total Sugar (gr)': 40/3, #https://www.livestrong.com/article/333407-how-many-calories-should-i-eat-at-lunch/
                        'Total lipid (Cal)': 0.176/3,
                        'Total Trans Fat (Cal)': .01/3, #https://medlineplus.gov/ency/patientinstructions/000786.htm#:~:text=You%20should%20limit%20saturated%20fat,or%202%20grams%20per%20day.
                        'Total Saturated Fat (Cal)': 0.1/3,
                        'Cholesterol (mg)' : 250/3, #https://www.healthline.com/health/high-cholesterol/rda#:~:text=Following%20dietary%20guidelines%2C%20doctors%20used,high%20risk%20of%20heart%20disease.
                        'Calcium (mg)': 1000/3,
                        'Iron (mg)': 8/2,
                        'Sodium (mg)': 2300/3,
                        'Vitamin A (mcg RAE)': 900/3,
                        'Vitamin C (mg)' : 90/3}
     

In [34]:
#Lists of DataFrames with the information of the required nutrients for each age and lifestyle goup.

maleInfo = []

for lifeStyle in ['Sedentary','Moderately active','Active']:
    
    lifeStyle_df = calorieNeeds_male[['AGE', lifeStyle]]
    
    for index,row in lifeStyle_df.iterrows():

        for key in nutritionParameters_dict.keys():

            if key not in lifeStyle_df.columns:

                lifeStyle_df[key] = ''

                if (key == 'Protein (Cal)') |( key == 'Carbohydrate (Cal)') | (key == 'Total Sugar (Cal)') | (key == 'Total lipid (Cal)')| (key == 'Total Trans Fat (Cal)')|( key == 'Total Saturated Fat (Cal)') | (key == 'Calories per Meal'):

                    lifeStyle_df.loc[index, key] = round(row[lifeStyle] * nutritionParameters_dict[key],2)

                else:

                    lifeStyle_df.loc[index, key] = round(nutritionParameters_dict[key],2)

            else:

                if (key == 'Protein (Cal)') |( key == 'Carbohydrate (Cal)') | (key == 'Total Sugar (Cal)') | (key == 'Total lipid (Cal)')| (key == 'Total Trans Fat (Cal)')|( key == 'Total Saturated Fat (Cal)') | (key == 'Calories per Meal'):

                    lifeStyle_df.loc[index, key] = round(row[lifeStyle] * nutritionParameters_dict[key],2)

                else:

                    lifeStyle_df.loc[index, key] = round(nutritionParameters_dict[key],2)

    columns = lifeStyle_df.columns.to_list()

    lifeStyle_df[columns] = lifeStyle_df[columns].apply(pd.to_numeric, axis=1) 

    lifeStyle_df = lifeStyle_df.groupby(lifeStyle).mean().reset_index().drop(['AGE'], axis=1)
                     
    maleInfo.append(lifeStyle_df)
                        
maleInfo[0] #List of DataFrames for males and the intake limits for each meal. 

,Sedentary,Protein (Cal),Carbohydrate (Cal),Fiber (gr),Total Sugar (Cal),Total lipid (Cal),Total Trans Fat (Cal),Total Saturated Fat (Cal),Cholesterol (mg),Calcium (mg),Iron (mg),"Sodium, (mg)",Vitamin A (mcg RAE),Vitamin C (mg),Calories per Meal
0,2000.0,150.0,350.0,19.0,80.0,200.0,20.0,60.0,100.0,582.5,10.0,460.0,615.0,21.0,666.67
1,2200.0,165.0,385.0,19.0,88.0,220.0,22.0,66.0,100.0,582.5,10.0,460.0,615.0,21.0,733.33
2,2400.0,180.0,420.0,19.0,96.0,240.0,24.0,72.0,100.0,582.5,10.0,460.0,615.0,21.0,800.00
3,2600.0,195.0,455.0,19.0,104.0,260.0,26.0,78.0,100.0,582.5,10.0,460.0,615.0,21.0,866.67


elif row['AGE'] > 19 & row['AGE'] <= 30 :

            for key in maleNutrition19to30_dict.keys():

                    if (key == 'Protein (Cal)') |( key == 'Carbohydrate (Cal)') | (key == 'Total lipid (Cal)')| (key == 'Total Trans Fat (Cal)')|( key == 'Total Saturated Fat (Cal)'):

                        lifeStyle_df.loc[index, key] = round(row[lifeStyle] * maleNutrition19to30_dict[key],2)

                    else:

                        lifeStyle_df.loc[index, key] = round(maleNutrition19to30_dict[key],2)

        elif row['AGE'] > 31 & row['AGE'] <= 50 :

            for key in maleNutrition31to50_dict.keys():

                    if (key == 'Protein (Cal)') |( key == 'Carbohydrate (Cal)') | (key == 'Total lipid (Cal)')| (key == 'Total Trans Fat (Cal)')|( key == 'Total Saturated Fat (Cal)'):

                        lifeStyle_df.loc[index, key] = round(row[lifeStyle] * maleNutrition31to50_dict[key],2)

                    else:

                        lifeStyle_df.loc[index, key] = round(maleNutrition31to50_dict[key],2)

        elif row['AGE'] > 31 & row['AGE'] <= 50 :

            for key in maleNutrition51_dict.keys():

                    if (key == 'Protein (Cal)') |( key == 'Carbohydrate (Cal)') | (key == 'Total lipid (Cal)')| (key == 'Total Trans Fat (Cal)')|( key == 'Total Saturated Fat (Cal)'):

                        lifeStyle_df.loc[index, key] = round(row[lifeStyle] * maleNutrition51_dict[key],2)

                    else:

                        lifeStyle_df.loc[index, key] = round(maleNutrition51_dict[key],2)

In [35]:
#Load kcal requirments from FDA

calorieNeeds_female = pd.read_excel('../../FDA Tables XLSX/Appendix 2. Estimated Calorie Needs -2020 Dietary Guidelines _ health.gov_0002.xlsx', sheet_name = 'Females')

calorieNeeds_female = calorieNeeds_female[calorieNeeds_female['AGE'] >= 14]

calorieNeeds_female =  calorieNeeds_female.reset_index(drop= True)

calorieNeeds_female.head()

,AGE,Sedentary,Moderately active,Active
0,14,1800,2000,2400
1,15,1800,2000,2400
2,16,1800,2000,2400
3,17,1800,2000,2400
4,18,1800,2000,2400


In [36]:
femaleInfo = []

for lifeStyle in ['Sedentary','Moderately active','Active']:
    
    lifeStyle_df = calorieNeeds_female[['AGE', lifeStyle]]
    
    for index,row in lifeStyle_df.iterrows():

        for key in nutritionParameters_dict.keys():

            if key not in lifeStyle_df.columns:

                lifeStyle_df[key] = ''

                if (key == 'Protein (Cal)') |( key == 'Carbohydrate (Cal)') | (key == 'Total Sugar (Cal)') | (key == 'Total lipid (Cal)')| (key == 'Total Trans Fat (Cal)')|( key == 'Total Saturated Fat (Cal)') | (key == 'Calories per Meal'):

                    lifeStyle_df.loc[index, key] = round(row[lifeStyle] * nutritionParameters_dict[key],2)

                else:

                    lifeStyle_df.loc[index, key] = round(nutritionParameters_dict[key],2)

            else:

                if (key == 'Protein (Cal)') |( key == 'Carbohydrate (Cal)') | (key == 'Total Sugar (Cal)') | (key == 'Total lipid (Cal)')| (key == 'Total Trans Fat (Cal)')|( key == 'Total Saturated Fat (Cal)') | (key == 'Calories per Meal'):

                    lifeStyle_df.loc[index, key] = round(row[lifeStyle] * nutritionParameters_dict[key],2)

                else:

                    lifeStyle_df.loc[index, key] = round(nutritionParameters_dict[key],2)
                    
    columns = lifeStyle_df.columns.to_list()

    lifeStyle_df[columns] = lifeStyle_df[columns].apply(pd.to_numeric, axis=1) 

    lifeStyle_df = lifeStyle_df.groupby(lifeStyle).mean().reset_index().drop(['AGE'], axis=1)
       
                        
    femaleInfo.append(lifeStyle_df)
                        
femaleInfo[0] #List of DataFrames for males and the intake limits for each meal. 

,Sedentary,Protein (Cal),Carbohydrate (Cal),Fiber (gr),Total Sugar (Cal),Total lipid (Cal),Total Trans Fat (Cal),Total Saturated Fat (Cal),Cholesterol (mg),Calcium (mg),Iron (mg),"Sodium, (mg)",Vitamin A (mcg RAE),Vitamin C (mg),Calories per Meal
0,1600.0,120.0,280.0,19.0,64.0,160.0,16.0,48.0,100.0,582.5,10.0,460.0,615.0,21.0,533.33
1,1800.0,135.0,315.0,19.0,72.0,180.0,18.0,54.0,100.0,582.5,10.0,460.0,615.0,21.0,600.00
2,2000.0,150.0,350.0,19.0,80.0,200.0,20.0,60.0,100.0,582.5,10.0,460.0,615.0,21.0,666.67


In [37]:
#Stores the DataFrames in the system to be abe to read it in another Notebook.

%store maleInfo

%store femaleInfo

%store foodFiltered_df

Stored 'maleInfo' (list)
Stored 'femaleInfo' (list)
Stored 'foodFiltered_df' (DataFrame)
